<a href="https://colab.research.google.com/github/Viasha/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [6]:
cleaned_df = df.dropna()
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [7]:
# Create the vine_table. DataFrame
vine_df = cleaned_df.select(["review_id","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8EWA1OFT84NX|          5|            0|          0|   N|                Y|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|                N|
| RL5ESX231LZ0B|          5|            0|          0|   N|                Y|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|                Y|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|                Y|
|R13EPSFP5DODN5|          4|            0|          0|   N|                Y|
| R6RBP4HTE67SY|          5|            0|          0|   N|                Y|
|R15B3EU40RSU2W|          5|            0|          0|   N|                Y|
| RP4DD53A4ZJA2|          5|            0|          0|   N|                Y|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|     

In [8]:
# Step 1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
|R3UUR313K5VVTL|          1|            1|         24|   N|                Y|
| RODE8K12S7148|          5|           41|         45|   N|     

In [9]:
# Step 2: Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
percent_votes_df = total_votes_df.withColumn('percent_votes',col('helpful_votes')/col('total_votes')).alias('percent_votes').filter(col("percent_votes") >= 0.5)
percent_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|0.9120879120879121|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|               1.0|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|0.9869451697127938|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|0.9642857142857143|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|0.8723404255319149|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|0.8333333333333334|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|0.9512195121951219|
|R3OIDSQJ84W7J1|          5|  

In [10]:
# Step 3: Retrieves all the rows where a review was written as part of the Vine program (paid)
paid_df = percent_votes_df.filter(col("vine") == "Y").show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|0.8723404255319149|
| RSA6JQ346JZHZ|          5|           55|         64|   Y|                Y|          0.859375|
|R1FXF4HRMCLG4C|          5|           69|         84|   Y|                N|0.8214285714285714|
| RCTBWC3II42MG|          4|          113|        117|   Y|                N|0.9658119658119658|
| RTMQM2CQ1XIZ0|          5|           21|         25|   Y|                N|              0.84|
|R2CBJLCKQ612KU|          4|           27|         35|   Y|                N|0.7714285714285715|
|R322QDGO4AV2B5|          5|           55|         65|   Y|                N|0.8461538461538461|
| R2S7CXRDGQ6EE|          4|  

In [11]:
# Step 4: Retrieves all the rows where the review was not part of the Vine program (unpaid)
non_paid_df = percent_votes_df.filter(col("vine") == "N").show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|0.9120879120879121|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|               1.0|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|0.9869451697127938|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|0.9642857142857143|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|0.8333333333333334|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|0.9512195121951219|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|             0.875|
| RODE8K12S7148|          5|  

In [12]:
# Step 5: Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

from pyspark.sql.functions import col,when,count,lit
ratings_total_df = percent_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_5_Star_To_Total")).show()

+----+-------------+--------------------+------------------+
|vine|Total_Reviews|Total_5_Star_Reviews| %_5_Star_To_Total|
+----+-------------+--------------------+------------------+
|   Y|          463|                 202|43.628509719222464|
|   N|        25079|               12028| 47.96044499381953|
+----+-------------+--------------------+------------------+

